# House Prices - Advanced Regression Techniques
## Predict sales prices and practice feature engineering, RFs, and gradient boosting

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview
    

## Part 1: Analyzing data

In [478]:
# Imports
import torch
import math, time
import numpy as np
import pandas as pd
from torch import nn
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

In [479]:
# Load data
test = pd.read_csv("data/test.csv")
train = pd.read_csv("data/train.csv")

In [480]:
test.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal
1458,2919,60,RL,74.0,9627,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,11,2006,WD,Normal


In [481]:
train.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [482]:
# We have 1 column more
train.columns.size, test.columns.size

(81, 80)

In [483]:
# So test doesn't have SalePrice
diff = set(train.columns) - set(test.columns)
diff

{'SalePrice'}

In [484]:
# So I assume the test set is for submitting the results
# I will have to use the train as my main dataset, with SalePrice as target value
dataset = train
X = dataset[test.columns]
y = dataset['SalePrice']
X.shape,y.shape

((1460, 80), (1460,))

In [485]:
# Are there missing values?
missing_values = X.columns[X.isnull().sum() > 0].tolist()
X[missing_values].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [486]:
y.isnull().sum()

0

In [487]:
# Lets see straight ahead what can we achieve when considering all the features.
# We'll ignore for now the features with missing values.

### Pre-process data

# Separate data Xm will be X without features with missing values
Xm = X.drop(columns=missing_values)
print(Xm.columns)

# Select the numerical columns
numerical_columns = Xm.select_dtypes(include=['int', 'float']).columns
X_num = Xm.loc[:, numerical_columns]

# Select the categorical columns
categorical_columns = Xm.select_dtypes(include=['object']).columns
X_cat = Xm.loc[:, categorical_columns]

print('Numerical columns:', numerical_columns.size)
print('Categorical columns:', categorical_columns.size)
print('Total:', categorical_columns.size + numerical_columns.size, " Valid: ", Xm.shape[1] == categorical_columns.size + numerical_columns.size)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageCars', 'GarageArea', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition'],
      dtype='object')
Numerical columns: 34
Categorical columns: 27
Total: 61  Vali

In [488]:
numerical_columns, categorical_columns

(Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
        'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
        'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
        'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'],
       dtype='object'),
 Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
        'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive',
        'SaleType', 'SaleCondition'],
       dtype='object'))

- Scaling numerical data is pretty straightforward, we have to standarize data so the data looks like a standart distribution which makes data comparable and so, machine learning estimators can behave as intended.
- Encoding categorical features is a bit more complicated, because we have to decide if we are going to encode it with an ordinal encoder or not. So for this experiment, I'm gonna go with just get dummies wich is for non ordinal encoding. Which means its going to create new columns with only 0 and 1, instead of turning the labels into a column with 0, 1, 2, 3... because if I do that, the model might learn that 3 > 1 which is not the case for example with colors.

In [489]:
# Preprocess the numerical features
scaler = StandardScaler()

X_num = scaler.fit_transform(X_num)

# Encode the categorical features
X_cat = pd.get_dummies(X_cat)

In [490]:
print(X_num.shape, X_cat.shape)

(1460, 34) (1460, 182)


In [491]:
X_merged = pd.concat([pd.DataFrame(X_num, columns=numerical_columns), pd.DataFrame(X_cat)], axis=1)
X_merged.shape

(1460, 216)

In [492]:
X_train, X_test, y_train, y_test = train_test_split(X_merged, y, test_size=0.2, random_state=0)

In [493]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1168, 216) (292, 216) (1168,) (292,)


In [494]:
print(y_train.dtype, y_test.dtype)

int64 int64


In [495]:
# Choose a regression algorithm
regressor = LinearRegression()

# Train the regression model
regressor.fit(X_train, y_train);

In [496]:
# Evaluate the performance of the model
y_pred = regressor.predict(X_test)
print(y_test.shape, y_test.shape)
results = pd.DataFrame(
    np.concatenate(
        (
            y_test.to_numpy().reshape(len(y_test), 1),
            y_pred.reshape(len(y_pred), 1)

        ), axis=1),
    columns=["Original", "Predicted"]
    )

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse:0.2f}')
print(f'Mean Absolute Error: {mae:0.2f}')

(292,) (292,)
Mean Squared Error: 4323736233256735788303908864.00
Mean Absolute Error: 8226226617921.66


In [497]:
# Baseline approach results
results[0:100], results[100:200], results[200:292]

(    Original  Predicted
 0   200624.0   275109.0
 1   133000.0   145210.0
 2   110000.0   117224.0
 3   192000.0   216340.0
 4    88000.0    98508.0
 ..       ...        ...
 95  157000.0   166554.0
 96  213000.0   213814.0
 97  185000.0   163470.0
 98  124500.0   106210.0
 99  162900.0   158950.0
 
 [100 rows x 2 columns],
      Original  Predicted
 100  260000.0   249340.0
 101  198500.0   236068.0
 102  120000.0   112726.0
 103  159500.0   152850.0
 104  105900.0   105946.0
 ..        ...        ...
 195  142500.0   150840.0
 196   80000.0    65056.0
 197  145000.0   144488.0
 198  171000.0   171700.0
 199  122500.0   118338.0
 
 [100 rows x 2 columns],
      Original  Predicted
 200  139000.0   116426.0
 201  189000.0   219712.0
 202  120500.0   114692.0
 203  124000.0   132612.0
 204  160000.0   110126.0
 ..        ...        ...
 287  324000.0   284284.0
 288  555000.0   460834.0
 289  136000.0   264300.0
 290   82500.0   127678.0
 291  101000.0   101224.0
 
 [92 rows x 2 column

- It shows a very big number for the MSE as well as MAE, and that's because the target data is in it's original price value, but if we look at the predictions, doesn't look so off the original value considering we haven't touched too much the data.
- So after we are done with cleaning most of the data, we'll start trying some feature selection techniques and other models.

## Part 2: Missing data
- We trained a simple regression model using all the data that had no missing values.
- But if we look at the missing data, there are some features that have only few missing parts that could be imputed so we have more features available.

In [498]:
X[missing_values].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [499]:
# Lets say we want to take into account these extra data
# Let's consider the ones with less than 100 missing points.

incomplete = X.columns[X.isnull().sum() > 0].tolist()
incomplete100 = X.columns[X.isnull().sum() <= 100].tolist()
incomplete_columns = list(set(incomplete) & set(incomplete100))
print(np.sort(incomplete_columns))

['BsmtCond' 'BsmtExposure' 'BsmtFinType1' 'BsmtFinType2' 'BsmtQual'
 'Electrical' 'GarageCond' 'GarageFinish' 'GarageQual' 'GarageType'
 'GarageYrBlt' 'MasVnrArea' 'MasVnrType']


In [500]:
X[incomplete_columns]

,GarageCond,BsmtExposure,BsmtFinType2,MasVnrArea,BsmtQual,MasVnrType,GarageYrBlt,GarageQual,GarageType,BsmtFinType1,BsmtCond,GarageFinish,Electrical
0,TA,No,Unf,196.0,Gd,BrkFace,2003.0,TA,Attchd,GLQ,TA,RFn,SBrkr
1,TA,Gd,Unf,0.0,Gd,None,1976.0,TA,Attchd,ALQ,TA,RFn,SBrkr
2,TA,Mn,Unf,162.0,Gd,BrkFace,2001.0,TA,Attchd,GLQ,TA,RFn,SBrkr
3,TA,No,Unf,0.0,TA,None,1998.0,TA,Detchd,ALQ,Gd,Unf,SBrkr
4,TA,Av,Unf,350.0,Gd,BrkFace,2000.0,TA,Attchd,GLQ,TA,RFn,SBrkr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,TA,No,Unf,0.0,Gd,None,1999.0,TA,Attchd,Unf,TA,RFn,SBrkr
1456,TA,No,Rec,119.0,Gd,Stone,1978.0,TA,Attchd,ALQ,TA,Unf,SBrkr
1457,TA,No,Unf,0.0,TA,None,1941.0,TA,Attchd,GLQ,Gd,RFn,SBrkr
1458,TA,Mn,Rec,0.0,TA,None,1950.0,TA,Attchd,GLQ,TA,Unf,FuseA


- Let's take a look at the descriptions for each one

1. 'BsmtCond': Evaluates the general condition of the basement
2. 'BsmtExposure': Refers to walkout or garden level walls
3. 'BsmtFinType1': Rating of basement finished area
4. 'BsmtFinType2': Rating of basement finished area (if multiple types)
5. 'BsmtQual': Height of the basement
6. 'Electrical': Electrical system
7. 'GarageCond': Garage condition
8. 'GarageFinish': Interior finish of the garage
9. 'GarageQual': Garage quality
10. 'GarageType': Garage location
11. 'GarageYrBlt': Year garage was built
12. 'MasVnrArea': Masonry veneer area in square feet
13. 'MasVnrType': Masonry veneer type
    
 
 - So at first glance, appears obvious that the missing data could be because the house doesn't have a basement or garage. The electrical system seems a bit less obvious one, and might be because of actual missing data because it's just 1 data that has no electrical system. I don't actually know about masonry veneer so I had to ask my friend ChatGPT which said:
 
   Masonry veneer is a type of exterior cladding that provides a **beautiful and durable finish** to a building's façade. There are several types of masonry veneer, including:

    **Brick veneer**: This is the most common type of masonry veneer. It is made up of a layer of bricks that are placed over a frame or wall. The bricks are typically held in place with mortar.

    **Stone veneer**: This type of veneer is made up of thin pieces of natural or manufactured stone that are attached to a frame or wall. Stone veneer can be used to achieve a wide range of aesthetic styles, from rustic to modern.

    **Stucco veneer**: Stucco is a mixture of cement, sand, and water that is applied to a wall or frame to create a smooth or textured finish. Stucco veneer can be colored and textured to mimic the appearance of other types of masonry.

    **Concrete block veneer**: Concrete blocks can be used as a veneer to achieve a modern, industrial look. The blocks can be left unfinished or painted to create a specific aesthetic.

    **Terra cotta veneer**: Terra cotta is a type of clay that can be molded into various shapes and sizes. Terra cotta veneer is a lightweight option that is often used for decorative features, such as window surrounds or cornices.

In [501]:
# Processing: MasVnrType & MasVnrArea
X['MasVnrType'].unique()

array(['BrkFace', 'None', 'Stone', 'BrkCmn', nan], dtype=object)

In [502]:
X[X['MasVnrType'].isnull()]['MasVnrArea']

234    NaN
529    NaN
650    NaN
936    NaN
973    NaN
977    NaN
1243   NaN
1278   NaN
Name: MasVnrArea, dtype: float64

- In this case we have in MasVnrType a category 'None' which we could use it to default the missing values, and for MasVnrArea we also can see in the data that some have 0.0, which could also work as a default value. We also don't have any more data about the masonry veneer to make us think that there is missing data and instead of 0 or None it could be another category, so best way would be set the values to default.
- Also happens that missing data from 'MasVnrArea' are the same rows, so we can default it to 0.

In [503]:
X.loc[X['MasVnrType'].isnull(), 'MasVnrType'] = 'None'
X.loc[X['MasVnrArea'].isnull(), 'MasVnrArea'] = 0.0
X['MasVnrType'].isnull().sum(), X['MasVnrType'].isnull().sum()

(0, 0)

In [504]:
# Processing: Electrical
X['Electrical'].value_counts(), X['Electrical'].unique()

(SBrkr    1334
 FuseA      94
 FuseF      27
 FuseP       3
 Mix         1
 Name: Electrical, dtype: int64,
 array(['SBrkr', 'FuseF', 'FuseA', 'FuseP', 'Mix', nan], dtype=object))

In [505]:
# Let's print this 1 row
X[X['Electrical'].isnull()], y[X['Electrical'].isnull()]
# We can see that it has the Utilities feature set to 'AllPub': All public Utilities (E,G,W,& S).
# I wouldn't mess much with only 1 datapoint
# so I'll set it to the average Electrical System: SBrkr Standard Circuit Breakers & Romex

(        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
 1379  1380          80       RL         73.0     9735   Pave   NaN      Reg   
 
      LandContour Utilities  ... ScreenPorch PoolArea PoolQC Fence MiscFeature  \
 1379         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   
 
      MiscVal MoSold  YrSold  SaleType  SaleCondition  
 1379       0      5    2008        WD         Normal  
 
 [1 rows x 80 columns],
 1379    167500
 Name: SalePrice, dtype: int64)

In [506]:
X.loc[X['Electrical'].isnull(), 'Electrical'] = 'SBrkr'
X['Electrical'].isnull().sum()

0

In [511]:
# Processing: Basement
bsmt_columns = ['BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual']
X[bsmt_columns].isnull().sum()

BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinType2    38
BsmtQual        37
dtype: int64

In [516]:
X[bsmt_columns][X['BsmtExposure'].isnull()]

,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual
17,NaN,NaN,NaN,NaN,NaN
39,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN
156,NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,NaN,NaN,NaN
259,NaN,NaN,NaN,NaN,NaN
342,NaN,NaN,NaN,NaN,NaN
362,NaN,NaN,NaN,NaN,NaN
371,NaN,NaN,NaN,NaN,NaN


- If we take a look, we can see that almost all the missing data from the basement comes from the same houses.
- We could just assume that those houses don't have a basement, but we have to look just in case there is some inconsistent data, for example, another feature that has a label which indicates the pressence of a basement.
- What other features consider a basement? and how it tells us if there is no basement?


In [525]:
for b in bsmt_columns:
    print(X[b].value_counts(), "\n")

TA    1311
Gd      65
Fa      45
Po       2
Name: BsmtCond, dtype: int64 

No    953
Av    221
Gd    134
Mn    114
Name: BsmtExposure, dtype: int64 

Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
Name: BsmtFinType1, dtype: int64 

Unf    1256
Rec      54
LwQ      46
BLQ      33
ALQ      19
GLQ      14
Name: BsmtFinType2, dtype: int64 

TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64 



- If we take a look at the data description we have that we have for example:

       BsmtExposure: Refers to walkout or garden level walls

       Gd	Good Exposure
       Av	Average Exposure (split levels or foyers typically score average or above)	
       Mn	Mimimum Exposure
       No	No Exposure
       NA	No Basement